In [11]:
%pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=20597 sha256=2de6be11066fc07bf9d7169452c64523d7c49e7a410d856676915be8c3740efb
  Stored in directory: /tmp/pip-ephem-wheel-cache-pbhirb11/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1


In [12]:
import torch 

device = torch.device("cuda:3")

In [13]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [14]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [15]:
from spuco.group_inference import JTTInference
from spuco.models import MLP
from spuco.util import Trainer
from torch.optim import SGD

model = MLP(3*28*28, len(classes)).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

jtt = JTTInference(trainset, trainer=trainer, num_epochs=1)

In [16]:
group_partition = jtt.infer_groups()

100%|██████████| 938/938 [00:00<00:00, 2232.16batch/s]


In [17]:
for key in group_partition.keys():
    print(key, len(group_partition[key]))

(0, 0) 59692
(0, 1) 308


In [18]:
from spuco.invariant_train import InvariantTrainsetWrapper
invariant_trainset = InvariantTrainsetWrapper(trainset, group_partition)

In [27]:
from torch.optim import SGD
from spuco.models import MLP
from spuco.invariant_train import GroupDRO, CustomSampleERM

model = MLP(3*28*28, len(classes)).to(device)
jtt_train = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    indices=group_partition[(0,0)] + 200 * group_partition[(0,1)],
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [28]:
jtt_train.train()

Epoch 0: 100%|██████████| 1896/1896 [00:03<00:00, 556.05batch/s, accuracy=91.66666666666667%, loss=0.345]


In [29]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 97.63593380614657
Group (0, 1) Test Accuracy: 48.93617021276596
Group (0, 2) Test Accuracy: 65.2482269503546
Group (0, 3) Test Accuracy: 54.846335697399525
Group (0, 4) Test Accuracy: 82.03309692671395
Group (1, 0) Test Accuracy: 81.66259168704157
Group (1, 1) Test Accuracy: 97.31051344743277
Group (1, 2) Test Accuracy: 73.2843137254902
Group (1, 3) Test Accuracy: 45.34313725490196
Group (1, 4) Test Accuracy: 31.61764705882353
Group (2, 0) Test Accuracy: 53.6
Group (2, 1) Test Accuracy: 36.266666666666666
Group (2, 2) Test Accuracy: 96.0
Group (2, 3) Test Accuracy: 37.6
Group (2, 4) Test Accuracy: 10.427807486631016
Group (3, 0) Test Accuracy: 38.44221105527638
Group (3, 1) Test Accuracy: 39.7984886649874
Group (3, 2) Test Accuracy: 55.91939546599496
Group (3, 3) Test Accuracy: 96.97732997481108
Group (3, 4) Test Accuracy: 50.377833753148614
Group (4, 0) Test Accuracy: 73.55163727959697
Group (4, 1) Test Accuracy: 23.173803526448363
Group (4, 2) Test Accurac

{(0, 0): 97.63593380614657,
 (0, 1): 48.93617021276596,
 (0, 2): 65.2482269503546,
 (0, 3): 54.846335697399525,
 (0, 4): 82.03309692671395,
 (1, 0): 81.66259168704157,
 (1, 1): 97.31051344743277,
 (1, 2): 73.2843137254902,
 (1, 3): 45.34313725490196,
 (1, 4): 31.61764705882353,
 (2, 0): 53.6,
 (2, 1): 36.266666666666666,
 (2, 2): 96.0,
 (2, 3): 37.6,
 (2, 4): 10.427807486631016,
 (3, 0): 38.44221105527638,
 (3, 1): 39.7984886649874,
 (3, 2): 55.91939546599496,
 (3, 3): 96.97732997481108,
 (3, 4): 50.377833753148614,
 (4, 0): 73.55163727959697,
 (4, 1): 23.173803526448363,
 (4, 2): 25.44080604534005,
 (4, 3): 57.57575757575758,
 (4, 4): 95.45454545454545}

In [30]:
evaluator.worst_group_accuracy

((2, 4), 10.427807486631016)

In [31]:
evaluator.average_accuracy

96.47618650456914

In [32]:
evaluator.evaluate_spurious_task()

47.29